In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import matplotlib.pyplot as plt
from torch import nn

%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# load the training and test datasets
transform_train = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(size=32, padding=4),
    transforms.ToTensor(),
    #transforms.Normalize((0.5), (0.5)) #Normalize to (-1,1)
])

transform_eval = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5), (0.5)) #Normalize to (-1,1)
])

trainset = torchvision.datasets.MNIST(root='data', train=True,download=True, transform=transform_train)
testset = torchvision.datasets.MNIST(root='data', train=False,download=True, transform=transform_eval)

In [4]:
#Define Dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [5]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc3_mean = nn.Linear(h_dim2, z_dim)
        self.fc3_std = nn.Linear(h_dim2, z_dim)
        # decoder 
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc3_mean(h), self.fc3_std(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std) #  same size as std, normal distribution with mean 0 and variance 1
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
VAE = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    VAE.to(device)

In [6]:
VAE

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3_mean): Linear(in_features=256, out_features=2, bias=True)
  (fc3_std): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [7]:
optimizer = torch.optim.Adam(VAE.parameters(),
                                 lr=0.01, 
                                 weight_decay=1e-5)
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [8]:
def train(epoch):
    VAE.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(trainloader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = VAE(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(trainloader.dataset)))

In [9]:
def test():
    VAE.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in testloader:
            data = data.cuda()
            recon, mu, log_var = VAE(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(testloader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [12]:
for epoch in range(1, 31):
    train(epoch)
    test()

C:\Users\asus\anaconda3\envs\myclone\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 546.174072
Train Epoch: 1 [6400/60000 (11%)]	Loss: 171.156006
Train Epoch: 1 [12800/60000 (21%)]	Loss: 160.211716
Train Epoch: 1 [19200/60000 (32%)]	Loss: 152.706070
Train Epoch: 1 [25600/60000 (43%)]	Loss: 161.347153
Train Epoch: 1 [32000/60000 (53%)]	Loss: 162.375565
Train Epoch: 1 [38400/60000 (64%)]	Loss: 158.782028
Train Epoch: 1 [44800/60000 (75%)]	Loss: 149.364517
Train Epoch: 1 [51200/60000 (85%)]	Loss: 150.578201
Train Epoch: 1 [57600/60000 (96%)]	Loss: 160.971161
====> Epoch: 1 Average loss: 164.7879
====> Test set loss: 157.0424
Train Epoch: 2 [0/60000 (0%)]	Loss: 158.106506
Train Epoch: 2 [6400/60000 (11%)]	Loss: 155.973495
Train Epoch: 2 [12800/60000 (21%)]	Loss: 153.721207
Train Epoch: 2 [19200/60000 (32%)]	Loss: 159.605270
Train Epoch: 2 [25600/60000 (43%)]	Loss: 151.420303
Train Epoch: 2 [32000/60000 (53%)]	Loss: 166.110062
Train Epoch: 2 [38400/60000 (64%)]	Loss: 156.298416
Train Epoch: 2 [44800/60000 (75%)]	Loss: 148.580856
Train Ep

Train Epoch: 15 [0/60000 (0%)]	Loss: 152.200317
Train Epoch: 15 [6400/60000 (11%)]	Loss: 145.422150
Train Epoch: 15 [12800/60000 (21%)]	Loss: 151.635590
Train Epoch: 15 [19200/60000 (32%)]	Loss: 154.967621
Train Epoch: 15 [25600/60000 (43%)]	Loss: 141.640335
Train Epoch: 15 [32000/60000 (53%)]	Loss: 148.948578
Train Epoch: 15 [38400/60000 (64%)]	Loss: 153.463181
Train Epoch: 15 [44800/60000 (75%)]	Loss: 139.895462
Train Epoch: 15 [51200/60000 (85%)]	Loss: 145.863510
Train Epoch: 15 [57600/60000 (96%)]	Loss: 148.903320
====> Epoch: 15 Average loss: 150.4988
====> Test set loss: 150.3100
Train Epoch: 16 [0/60000 (0%)]	Loss: 146.384979
Train Epoch: 16 [6400/60000 (11%)]	Loss: 149.890198
Train Epoch: 16 [12800/60000 (21%)]	Loss: 153.281982
Train Epoch: 16 [19200/60000 (32%)]	Loss: 152.172440
Train Epoch: 16 [25600/60000 (43%)]	Loss: 147.262161
Train Epoch: 16 [32000/60000 (53%)]	Loss: 143.279327
Train Epoch: 16 [38400/60000 (64%)]	Loss: 153.527664
Train Epoch: 16 [44800/60000 (75%)]	Loss: 

====> Epoch: 28 Average loss: 150.1341
====> Test set loss: 150.9705
Train Epoch: 29 [0/60000 (0%)]	Loss: 148.369888
Train Epoch: 29 [6400/60000 (11%)]	Loss: 152.547699
Train Epoch: 29 [12800/60000 (21%)]	Loss: 151.674133
Train Epoch: 29 [19200/60000 (32%)]	Loss: 161.132309
Train Epoch: 29 [25600/60000 (43%)]	Loss: 146.740417
Train Epoch: 29 [32000/60000 (53%)]	Loss: 155.086121
Train Epoch: 29 [38400/60000 (64%)]	Loss: 153.117676
Train Epoch: 29 [44800/60000 (75%)]	Loss: 148.310074
Train Epoch: 29 [51200/60000 (85%)]	Loss: 151.034760
Train Epoch: 29 [57600/60000 (96%)]	Loss: 155.261719
====> Epoch: 29 Average loss: 150.9881
====> Test set loss: 150.5300
Train Epoch: 30 [0/60000 (0%)]	Loss: 153.597595
Train Epoch: 30 [6400/60000 (11%)]	Loss: 157.713623
Train Epoch: 30 [12800/60000 (21%)]	Loss: 159.840271
Train Epoch: 30 [19200/60000 (32%)]	Loss: 151.402115
Train Epoch: 30 [25600/60000 (43%)]	Loss: 149.498840
Train Epoch: 30 [32000/60000 (53%)]	Loss: 145.843353
Train Epoch: 30 [38400/600